In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

print("This is as of " + str(datetime.now()))

This is as of 2017-08-08 18:31:13.586059


In [ ]:
# Get the latest data from the Chicago data portal right now
%run "Get beach_days from API.ipynb"
beach_days.head()

Deleting columns: 


In [ ]:
# How many days of samples were taken
print("Samples were taken across %s days" % len(beach_days['date'].unique()))

In [ ]:
print("There are %s total beach-days, and %s unique predictions" %
     (str(len(beach_days)), str(len(beach_days.loc[~beach_days['predicted_level'].isnull()]))))

In [ ]:
beach_days['elevated'] = beach_days['dna_reading_mean'] >= 1000
beach_days['predicted_elevated'] = beach_days['predicted_level'] >=250

beach_days.head(10)

In [ ]:
predicted_days = beach_days.loc[~beach_days['predicted_level'].isnull()]
len(predicted_days)

In [ ]:
measured_days = beach_days.loc[~beach_days['dna_reading_mean'].isnull()]
len(measured_days)

In [ ]:
days_with_both = predicted_days.loc[~predicted_days['dna_reading_mean'].isnull()]
len(days_with_both)

In [ ]:
a = pd.DataFrame()
a = a.append(
    [['TP (elevated, predicted elevated)',
         len(days_with_both.loc[(days_with_both['predicted_elevated']) & (days_with_both['elevated'])])],
         ['TN (elevated, but predicted not)',
         len(days_with_both.loc[(days_with_both['predicted_elevated'] == 0) & (days_with_both['elevated'])])],
         ['FP (not elevated, but predicted so)',
         len(days_with_both.loc[(days_with_both['predicted_elevated']) & (0 == days_with_both['elevated'])])],
         ['FN (not elevated, not predicted so)',
         len(days_with_both.loc[(0 == days_with_both['predicted_elevated']) & (0 == days_with_both['elevated'])])]
    ]
)
a.columns = ['label', 'days']
a = a.set_index('label')
a['days'].plot(kind="pie", title="Accuracy of predictions", autopct='%1.1f%%')

In [ ]:
# For the DNA testing, the threshold for an advisory is 1000 CFU.  How many beaches exceed that?
beach_days.loc[beach_days['dna_reading_mean'] >= 1000].groupby('beach_name').count()['date']\
    .sort_values(ascending=False).plot(kind="bar", title="Samples above 1000 cfu")

In [ ]:
beach_days.loc[beach_days['dna_reading_mean'] >= 1000].groupby('date').count()['location']\
    .plot(kind="bar", title="Days with samples above 1000 cfu").set_ylabel("Numbrt of samples exceeding threshold")

In [ ]:
# Samples that exceed threshold
beach_days['dna_exceeds'] = beach_days['dna_reading_mean'] > 1000
beach_days.groupby('dna_exceeds').count()['date'].plot(kind="pie", title="Percent of samples taken that exceed 1000 cfu", autopct='%1.1f%%')

In [ ]:
# What have been the highest readings so far?
beach_days.sort_values('dna_reading_mean', ascending=False)[['beach_name', 'date', 'dna_reading_mean']].head(20)